In [1]:
import numpy as np
import pandas as pd

from obj.Compos_DF import ComposDF
from obj.Compo_HTML import *
from obj.List import *
from obj.Block import *
from obj.Group import *
from obj.React import *
from obj.Page import *
from obj.Tree import *
import lib.draw as draw
from lib.list_item_gethering import gather_lists_by_pair_and_group

In [2]:
name = 'data/input/9'
compos = ComposDF(json_file=name + '-new.json', img_file=name + '.png')
img = compos.img.copy()
img_shape = compos.img_shape
img_re = cv2.resize(img, img_shape)

In [3]:
# ***Step 1*** repetitive list recognition
compos.repetitive_group_recognition()    # group_nontext, group_text
# check_valid_group_by_interleaving(compos.compos_dataframe)
compos.pair_groups()                     # group_pair, pair_to, group
# compos.list_item_partition()             # list_item
# compos.visualize_block('group')
# compos.visualize_block('group_pair')

In [4]:
c = compos.compos_dataframe

In [9]:
compos.visualize_block('cluster_row_min')

In [7]:
grps = c.groupby('cluster_row_min')

In [8]:
g = grps.groups
g[0]

Int64Index([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [59]:
for k in grps.groups:
    g = grps.groups[k]
    gaps = []
    for i in range(len(g) - 1):
        c_id = g[i]
        gaps.append(c.loc[g[i + 1]]['column_min'] - c.loc[g[i]]['column_min'])

    l_dic = {}
    for i,g in enumerate(gaps):
        grouped = False
        for key in l_dic:
            # valid grouped, update mean in the group
            if g < key < g*2 or key < g < key*2:
                l_dic[key].append(i)
                l_dic[np.mean((g, key))] = l_dic[key]
                del l_dic[key]
                grouped = True
                break
        if not grouped:
            l_dic[g] = [i]
    print(l_dic)

{40.78125: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 161: [10]}
{}
{}
{49.0625: [0, 1, 2, 3, 4, 5]}
{}
{}
{62.5: [0, 1]}
{82.625: [0, 1, 2, 3]}
{}
{}
{}
{}
{}
{}
{326: [0]}
{}
{}
{}
{}
{-566: [28], -543: [5], 4: [0], 0: [37], -121: [22], -472: [12], 9.5: [23, 25, 27], -437: [33], 172.375: [7, 17, 21, 38], -80: [20], -46: [18], -141: [10], 48.421875: [4, 11, 13, 14, 15, 16, 19, 24, 26], 348.203125: [1, 3, 6, 9, 29, 32, 39], -580: [8], -483: [2], -193: [40]}


In [60]:
compos.visualize('id')

In [42]:
l_dic

{40.78125: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 161: [10]}

In [5]:
c

,column_min,row_min,column_max,row_max,id,width,clip_path,height,class,area,...,cluster_center_column,cluster_center_row,cluster_area,alignment_in_group,cluster_row_min,cluster_column_min,group,pair_to,group_pair,list_item
0,0,0,635,800,0,635,data/output\clips\bkg.png,800,Background,508000,...,0.0,0.0,0.0,-1,-1.0,-1.0,-1,-1,-1,-1
1,4,0,12,9,1,8,data/output\clips\Compo\0.jpg,9,Compo,72,...,1.0,1.0,1.0,h,-1.0,-1.0,nt-10,-1,-1,-1
2,13,2,29,9,2,16,data/output\clips\Text\1.jpg,7,Text,112,...,-1.0,-1.0,-1.0,h,0.0,0.0,t-0,-1,-1,-1
3,45,3,56,9,3,11,data/output\clips\Text\2.jpg,6,Text,66,...,-1.0,-1.0,-1.0,h,0.0,1.0,t-0,-1,-1,-1
4,73,3,80,9,4,7,data/output\clips\Text\3.jpg,6,Text,42,...,-1.0,-1.0,-1.0,h,0.0,2.0,t-0,-1,-1,-1
5,95,1,110,9,5,15,data/output\clips\Text\4.jpg,8,Text,120,...,-1.0,-1.0,-1.0,h,0.0,3.0,t-0,-1,-1,-1
6,125,2,147,9,6,22,data/output\clips\Text\5.jpg,7,Text,154,...,-1.0,-1.0,-1.0,h,0.0,4.0,t-0,-1,-1,-1
7,163,1,178,9,7,15,data/output\clips\Text\6.jpg,8,Text,120,...,-1.0,-1.0,-1.0,h,0.0,5.0,t-0,-1,-1,-1
8,193,1,215,9,8,22,data/output\clips\Text\7.jpg,8,Text,176,...,-1.0,-1.0,-1.0,h,0.0,6.0,t-0,-1,-1,-1
9,232,3,269,9,9,37,data/output\clips\Text\8.jpg,6,Text,222,...,-1.0,-1.0,-1.0,h,0.0,7.0,t-0,-1,-1,-1


In [10]:
# ***Step 2*** mark compos in same group as a single list, mark compos in same group_pair as a multiple list
lists, non_listed_compos = gather_lists_by_pair_and_group(compos.compos_dataframe[1:])
generate_lists_html_css(lists)
visualize_lists(img_re, lists)

In [11]:
# ***Step 3*** slice compos as blocks
compos_html = [li.list_obj for li in lists] + non_listed_compos
blocks, non_blocked_compos = slice_blocks(compos_html, 'v')
# visualize_CompoHTMLs(compos_html, img_re)
# visualize_blocks(blocks, img, img_shape)
# blocks[2].visualize_sub_blocks_and_compos(img_re, recursive=True)

In [12]:
# ***Step 4*** assembly html and css as web page, and react program
html, css = export_html_and_css(blocks)
blk, index = export_react_program(blocks)
tree = export_tree(blocks)